# Utils

In [ ]:
# default_exp utils

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export
from cfnet.import_essentials import *

In [ ]:
# export
def validate_configs(
    configs: Union[Dict[str, Any], BaseParser], # configs
    config_class: BaseParser # config class
):
    if not isinstance(configs, config_class):
        configs = config_class(**configs)
    return configs

In [ ]:
# export
def load_json(f_name: str) -> Dict[str, Any]:
    with open(f_name) as f: return json.load(f)